<a href="https://colab.research.google.com/github/Laura-Neff/HandlingMulticollinearity/blob/main/HandlingMulticollinearity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd

In [80]:
automobile = pd.read_csv('cars_processed.csv')

automobile.head(5)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Origin,Age
0,18.000000,8,307.0,130,3504,12.0,US,49
1,16.000000,8,304.0,150,3433,12.0,US,49
2,17.000000,8,302.0,140,3449,10.5,US,49
3,14.000000,8,454.0,220,4354,9.0,US,49
4,23.551429,8,440.0,215,4312,8.5,US,49


In [81]:
automobile.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000
mean,23.672514,5.410853,192.184755,103.645995,2965.387597,15.573643,42.917313
std,7.736579,1.667795,103.703706,38.128651,846.332848,2.746260,3.668715
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,37.000000
25%,17.600000,4.000000,102.500000,75.000000,2221.500000,13.900000,40.000000
50%,23.200000,4.000000,146.000000,92.000000,2790.000000,15.500000,43.000000
75%,29.000000,6.000000,260.000000,121.000000,3589.500000,17.050000,46.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,49.000000


In [82]:
from sklearn import preprocessing

#scale data to center mean around 0 because all our stds and means are different

automobile[['Cylinders']] = preprocessing.scale(automobile[['Cylinders']].astype('float64'))
automobile[['Displacement']] = preprocessing.scale(automobile[['Displacement']].astype('float64'))
automobile[['Horsepower']] = preprocessing.scale(automobile[['Horsepower']].astype('float64'))
automobile[['Weight']] = preprocessing.scale(automobile[['Weight']].astype('float64'))
automobile[['Acceleration']] = preprocessing.scale(automobile[['Acceleration']].astype('float64'))
automobile[['Age']] = preprocessing.scale(automobile[['Age']].astype('float64'))

In [83]:
automobile.describe()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
count,387.000000,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02,3.870000e+02
mean,23.672514,3.672055e-17,1.101617e-16,1.101617e-16,1.101617e-16,2.203233e-16,8.078522e-16
std,7.736579,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00,1.001294e+00
min,9.000000,-1.447404e+00,-1.199046e+00,-1.513838e+00,-1.600007e+00,-2.761372e+00,-1.615000e+00
25%,17.600000,-8.470340e-01,-8.659368e-01,-7.522710e-01,-8.800918e-01,-6.102152e-01,-7.962160e-01
50%,23.200000,-8.470340e-01,-4.459295e-01,-3.058349e-01,-2.075007e-01,-2.685060e-02,2.256768e-02
75%,29.000000,3.537065e-01,6.547792e-01,4.557326e-01,7.383860e-01,5.382839e-01,8.413513e-01
max,46.600000,1.554447e+00,2.537570e+00,3.318176e+00,2.572779e+00,3.363956e+00,1.660135e+00


In [84]:
automobile.shape

(387, 8)

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
X = automobile.drop(['MPG', 'Origin'], axis=1)
Y = automobile['MPG']

#MPG is the label, so drop it and then train model. Leave Origin out for convenience because it contains categorical values.
#If you wanted to, you can one-hot encode that and use it as well

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [87]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(normalize=True).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


In [88]:
print("Training_score : " , linear_model.score(x_train, y_train))
#Pretty good

Training_score :  0.7800466519834268


In [89]:
y_pred = linear_model.predict(x_test)

In [90]:
from sklearn.metrics import r2_score

print("Testing_score : ", r2_score(y_test, y_pred))
#seeing our score for test is even higher than our score for training!

Testing_score :  0.8022844249155991


In [91]:
#when you have multiple predictors or multiple features in your regression model
#a better measure of how good your model is the adjusted r^2vscore
#Look below to see how we do that

#The adjusted r^2 score is calculated from the r^2 score and is a corrected goodness of fit measure for linear models
#this is an r^2 that has been adjusted for the number of predictors that you've used for your regression analysis 
#the adjusted r^2 only increases if a new predictor you add to train your model improves your model more than the improvement
#that can be expected due to chance

def adjusted_r2(r_square, labels, features):
    
    adj_r_square = 1 - ((1 - r_square) * (len(labels) - 1)) / (len(labels) - features.shape[1] - 1)
    
    return adj_r_square

In [92]:
print("Adjusted_r2_score : ", adjusted_r2(r2_score(y_test, y_pred), y_test, x_test))
#went down a little bit 

Adjusted_r2_score :  0.7855760664577625


In [93]:
features_corr = X.corr()

features_corr

#if you look at the correlation matrix, you see how Cylinders, Horsepower, and Weight are highly correlated with Displacement
#which suggests they are colinear. Another way to look at it is that they all give you the same information, so there's no reason to use all of it for your regression analysis

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
Cylinders,1.000000,0.922633,0.811466,0.873029,-0.458161,0.321850
Displacement,0.922633,1.000000,0.894199,0.932822,-0.526901,0.357047
Horsepower,0.811466,0.894199,1.000000,0.863388,-0.670920,0.404458
Weight,0.873029,0.932822,0.863388,1.000000,-0.397181,0.299049
Acceleration,-0.458161,-0.526901,-0.670920,-0.397181,1.000000,-0.292705
Age,0.321850,0.357047,0.404458,0.299049,-0.292705,1.000000


In [94]:
abs(features_corr) > 0.8

#Here we can figure out that cylinders, displacement, horsepower and weight are correlated


,Cylinders,Displacement,Horsepower,Weight,Acceleration,Age
Cylinders,True,True,True,True,False,False
Displacement,True,True,True,True,False,False
Horsepower,True,True,True,True,False,False
Weight,True,True,True,True,False,False
Acceleration,False,False,False,False,True,False
Age,False,False,False,False,False,True


In [95]:
#So we are dropping 'cylinders', 'displacement', 'weight' column
trimmed_features_df = X.drop(['Cylinders', 'Displacement', 'Weight'], axis=1)

In [96]:
trimmed_features_corr = trimmed_features_df.corr()

trimmed_features_corr

,Horsepower,Acceleration,Age
Horsepower,1.000000,-0.670920,0.404458
Acceleration,-0.670920,1.000000,-0.292705
Age,0.404458,-0.292705,1.000000


In [97]:
abs(trimmed_features_corr) > 0.8

,Horsepower,Acceleration,Age
Horsepower,True,False,False
Acceleration,False,True,False
Age,False,False,True


Calculating VIF score for multicollinearity detection
- 1 = not correlated.
- Between 1 and 5 = moderately correlated.
- Greater than 5 = highly correlated

In [98]:
#The variation inflation factor is a measure to quantify the severity of multicollinearity in an ordinary least square regression analysis

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [99]:
#The VIF for a particular feature i is calculated as a relationship between all other features, that is all features other than i and that feature i

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [100]:
vif["features"] = X.columns

In [101]:
vif.round(2) #round to 2 decimal places

,VIF Factor,features
0,6.84,Cylinders
1,16.10,Displacement
2,8.82,Horsepower
3,10.69,Weight
4,2.49,Acceleration
5,1.22,Age


In [102]:
X = X.drop(['Displacement', 'Weight'], axis=1)

In [103]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [104]:
vif["features"] = X.columns

In [112]:
vif.round(2)

,VIF Factor,features
0,3.05,Cylinders
1,4.56,Horsepower
2,1.90,Acceleration
3,1.20,Age


Here we are dropping that features which are causing multicollinearity and then training the model.
- Here we can see the difference between training, testing and adjusted r2 scores between the models that we build in the starting of the demo and this model.

In [106]:
X = automobile.drop(['MPG', 'Displacement', 'Weight', 'Origin'], axis=1) #because they are highly correlated 
Y = automobile['MPG']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [107]:
linear_model = LinearRegression(normalize=True).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


In [108]:
print("Training_score : " , linear_model.score(x_train, y_train))

Training_score :  0.7160292747712603


In [109]:
y_pred = linear_model.predict(x_test)

In [110]:
print("Testing_score : ", r2_score(y_test, y_pred))

Testing_score :  0.7608679096203809


In [111]:
print("Adjusted_r2_score : ", adjusted_r2(r2_score(y_test, y_pred), y_test, x_test))
#supposed to be less gap between this in the previous than before because we got rid of highly correlated factors

Adjusted_r2_score :  0.7477647813804018
